# Выделение списка условий из вакансии

In [1]:
import pandas as pd
data = pd.read_csv('data_forprint.csv',encoding='utf8',index_col=None)
data= data.drop('Unnamed: 0',axis=1)
data.head(5)

,domain_id,domain_name,profearea_id,profearea_name,ad_id,name,requirements,responsibilities,conditions
0,1,"Информационные технологии, интернет, телеком",1.395,Банковское ПО,35549307,Директор проектов (Лаборатория технологий омни...,Опыт работы не менее 5 лет в руководящей позиц...,Управление проектами в области цифровых канало...,Место работы: м. Киевская \nГрафик работы 5/2 ...
1,1,"Информационные технологии, интернет, телеком",1.327,Управление проектами,35549307,Директор проектов (Лаборатория технологий омни...,Опыт работы не менее 5 лет в руководящей позиц...,Управление проектами в области цифровых канало...,Место работы: м. Киевская \nГрафик работы 5/2 ...
2,1,"Информационные технологии, интернет, телеком",1.395,Банковское ПО,35966411,Начальник отдела стратегических проектов и тех...,Умение логически выстраивать бизнес-процессы;...,Инициирование новых доработок (подготовка бизн...,Социальный пакет с расширенным ДМС со стоматол...
3,1,"Информационные технологии, интернет, телеком",1.395,Банковское ПО,36012308,Project Manager (Digital),Высшее техническое или высшее экономическое об...,Реализация проектов по созданию digital-процес...,"Профессиональное обучение, семинары, тренинги,..."
4,1,"Информационные технологии, интернет, телеком",1.221,"Программирование, Разработка",36012308,Project Manager (Digital),Высшее техническое или высшее экономическое об...,Реализация проектов по созданию digital-процес...,"Профессиональное обучение, семинары, тренинги,..."


In [2]:
from east.asts import base
import operator
import numpy as np
import re

In [3]:
unique_data = data[['ad_id','conditions']]
unique_data=unique_data.drop_duplicates()
unique_data.head(5)

,ad_id,conditions
0,35549307,Место работы: м. Киевская \nГрафик работы 5/2 ...
2,35966411,Социальный пакет с расширенным ДМС со стоматол...
3,36012308,"Профессиональное обучение, семинары, тренинги,..."
7,35754828,Работа в молодой команде крутых специалистов; ...
8,36000461,Офис на м. Кутузовская.\nДостойная заработная ...


## Cписок выделенных условий работы

In [4]:
pat = re.compile(r'[^A-Za-z0-9 \-\n\r.,;!?А-Яа-я]+')
unique_data.conditions = [re.sub(pat,' ',x.replace('\n',' ')).lower() for x in unique_data.conditions.values[::]]

In [5]:
data_cond = unique_data.conditions[::].values
idxs = unique_data.ad_id[::].values
f = open('cond_picked.txt', 'r')
cond_picked = f.readlines()
f.close()
data_cond= list(data_cond)

In [6]:
data_cond[0:4]

['место работы  м. киевская  график работы 5 2  зп обсуждается  отличные возможности для профессионального развития и самореализации  расширенный соц. пакет  дмс, льготные условия по продуктам банка и страховой компании, корпоративные скидки в спортивные клубы   корпоративные скидки на страховые продукты  автострахование, взр и т.д.   специальные мероприятия для детей сотрудников  работа в крупной, стабильной компании ',
 'социальный пакет с расширенным дмс со стоматологией, который полностью компенсируется банком;  льготы на продукты и услуги родного банка  депозит с повышенной ставкой, выгодные кредиты по сниженной ставке, льготные условия по аренде сейфовых ячеек;   зарплатная карта с кешбэком;    уютный офис в историческом районе москвы, куда мы с радостью ид м даже утром в понедельник;    комфортное рабочее место. мы ценим личное пространство, поэтому сделали для сотрудников кабинеты, а не опенспейсы;    дружелюбная атмосфера и открытая корпоративная культура, в основе которой   с

## Построение АСД для оценки релевантности выделенных фраз

In [7]:
ast = base.AST.get_ast(data_cond)

In [8]:
cond_picked = list(map(lambda x: re.sub(pat,' ',x.replace('\n',' ')).lower(), cond_picked))

In [9]:
cond_ast = {}
for el in cond_picked:
    try:
        cond_ast[el] = ast.score(el)
    except:
        print('oops')
        print(el)

In [10]:
cond_ast = reversed(sorted(cond_ast.items(), key=operator.itemgetter(1)))
for el in cond_ast:
    print(el)

('высококвалифицированный коллектив ', 0.5267778450083754)
('дисконт-программы ', 0.5244170888053665)
('официальное трудоустройство ', 0.4933139671682926)
('высококвалифицированная команда ', 0.49093711067151335)
('корпоративные мероприятия ', 0.4585429859034907)
('самореализация ', 0.45528181835365794)
('конкурентоспособная зарплата ', 0.44812568570828204)
('страхование ', 0.4411650953024338)
('профессиональная команда ', 0.4394090403340437)
('корпоративная библиотека ', 0.4391628778089146)
('профессиональное развитие ', 0.4344391525708004)
('профессиональный рост ', 0.4242440934010127)
('доброжелательная атмосфера ', 0.4150891476877966)
('индивидуальный график ', 0.4146859146753325)
('сертификация ', 0.4080889523582796)
('повышение квалификации ', 0.40201993159911914)
('финансирование обучения ', 0.39359936711068094)
('дополнительный отпуск ', 0.38735544140035527)
('обучение ', 0.3804419216229543)
('вакцинация ', 0.37629018424058136)
('льготное кредитование ', 0.3734236145067381)
('с

## Построение матрицы релевантности вакансия - фраза из "очищенного" списка условий работы

Очищенный список - список, из которого удалены фразы с релевантностью меньше 0.1

In [11]:
f = open('cond_picked_clean.txt', 'r')
cond_picked = f.readlines()
f.close()
data_cond= list(data_cond)
cond_picked = list(map(lambda x: re.sub(pat,' ',x.replace('\n',' ')).lower(), cond_picked))

In [12]:
def make_substrings(tokens, k=4):
    tokens = tokens.split()
    for i in range(max(len(tokens) - k + 1, 1)):
        yield ' '.join(tokens[i:i + k])
        
def get_relevance_matrix(texts, strings):

    matrix = np.empty((0, len(strings)), float)

    for text in texts:
        ast = base.AST.get_ast(list(make_substrings(text)))
        row = np.array([ast.score(s) for s in strings])
        matrix = np.append(matrix, [row], axis=0)

    return matrix

In [13]:
relevance_matrix = get_relevance_matrix(data_cond, cond_picked)
print("Relevance mairix:")

Relevance mairix:


In [14]:
np.savetxt('relevance_matrix_cond.txt',relevance_matrix)

In [15]:
mask = relevance_matrix>=0.3
idx_x_conditions = pd.DataFrame(columns=['ad_id','conditions_refined','cond_idxs'])
for i in range(len(mask)):
    idx_x_conditions = idx_x_conditions.append({'ad_id': idxs[i],'conditions_refined':np.array(cond_picked)[mask[i]],'cond_idxs':np.where(relevance_matrix[i]>=0.3)}, ignore_index=True)
    print(np.array(cond_picked)[mask[i]])

['профессиональное развитие ' 'профессиональная команда '
 'профессиональный рост ' 'личное развитие ' 'скидки '
 'корпоративные мероприятия ' 'самореализация ' 'страхование '
 'стабильная компания ' 'мероприятия для детей ' 'социальный пакет '
 'возможность влиять ']
['корпоративная библиотека ' 'социальный пакет ' 'зарплатная карта '
 'комфортный офис ']
['участие в крупных проектах ' 'обучение ' 'профессиональное развитие '
 'профессиональная команда ' 'дружный коллектив ' 'профессиональный рост '
 'личное развитие ' 'посещение конференций ' 'интересные проекты '
 'бесплатное обучение ' 'развивающаяся компания ' 'социальный пакет '
 'тренинги ' 'возможность влиять ' 'фитнес-зал ']
['оформление по тк ' 'обучение ' 'индивидуальный график '
 'развивающаяся компания ' 'возможность влиять ' 'комфортный офис ']
['оформление по тк ' 'льготное кредитование '
 'достойная заработная плата ' 'участие в крупных проектах ' 'обучение '
 'английский ' 'дисконт-программы ' 'сертификация ' 'бесплатн

In [16]:
final = pd.merge(data, idx_x_conditions, left_on='ad_id', right_on='ad_id', how='left')
final[['conditions','conditions_refined']]
final[['profearea_name','ad_id','conditions','conditions_refined','cond_idxs']].to_csv('data_with_cond_refined.csv')

In [17]:
relevance_matrix.size

88270

In [18]:
np.set_printoptions(threshold=np.inf, linewidth=np.inf)  # turn off summarization, line-wrapping
with open('relevance_matrix_cond_2.txt', 'w') as f:
    f.write(np.array2string(relevance_matrix, separator=', '))

In [19]:
np.savetxt('relevance_matrix_cond_2.txt',relevance_matrix)